In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
device = 'cuda:0'

### model

In [3]:
import torch
model, transform = torch.hub.load("harvard-visionlab/open_ipcl", "alexnetgn_supervised_ref13_augset1_1x")
model = model.to(device)

Using cache found in /home/chris/.cache/torch/hub/harvard-visionlab_open_ipcl_master
/home/chris/miniconda3/envs/circuit_pruner/lib/python3.7/site-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla K40c with CUDA capability sm_35 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the Tesla K40c GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


### data loader

We want to pass imagenet data through the model, Im not sure how you access that data, but you need the official Imagenet dset on your file system, with 'train' & 'val' subfolders, with category-wise subfolders 'n01440764', 'n01443537' etc. point the following 'data_folder' variable to that imagenet path.

In [4]:
data_folder = '/mnt/data/datasets/imagenet/val/'

In [6]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

kwargs = {'num_workers': 4, 'pin_memory': True, 'sampler':None} if 'cuda' in device else {}

dset = ImageFolder(data_folder,transform=transform)
dloader = DataLoader(dset,
                     batch_size=256,
                     shuffle=False,
                     **kwargs)


### Prepping model

We want to 'score' the features in a layer by how much they might affect the model loss. We can approximate that as the average activationxgrad passing through a feature. Intuitively, this works because the gradient measures how much 'changing' the feature would affect the loss, while the activation size measures how much the feature would change (setting a high activation feature to 0 activation is a big change). Well use a 'hook' to save activation/gradient values.

In [7]:
model

alexnet_gn(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2), bias=False)
    (1): GroupNorm(32, 96, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (1): GroupNorm(32, 256, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): GroupNorm(32, 384, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
  )
  (conv_block_4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): GroupNorm(32, 384, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
  )
  (conv_blo

In [8]:
#little trick for getting a dictionary with reference names for each module in your model, at all nestings
layers = dict([*model.named_modules()])
#so now we can reference modules with a string;
print(layers.keys())
layers['fc6.0']

dict_keys(['', 'conv_block_1', 'conv_block_1.0', 'conv_block_1.1', 'conv_block_1.2', 'conv_block_1.3', 'conv_block_2', 'conv_block_2.0', 'conv_block_2.1', 'conv_block_2.2', 'conv_block_2.3', 'conv_block_3', 'conv_block_3.0', 'conv_block_3.1', 'conv_block_3.2', 'conv_block_4', 'conv_block_4.0', 'conv_block_4.1', 'conv_block_4.2', 'conv_block_5', 'conv_block_5.0', 'conv_block_5.1', 'conv_block_5.2', 'conv_block_5.3', 'ave_pool', 'fc6', 'fc6.0', 'fc6.1', 'fc6.2', 'fc7', 'fc7.0', 'fc7.1', 'fc7.2', 'fc8', 'fc8.0'])


Linear(in_features=9216, out_features=4096, bias=True)

In [11]:
dict([*model.named_modules()])['fc6.0']

Linear(in_features=9216, out_features=4096, bias=True)

In [12]:
from torch import nn, Tensor
from typing import Dict, Iterable, Callable

class actgrad_extractor(nn.Module):
    def __init__(self, model: nn.Module, layers: Iterable[str]):
        super().__init__()
        self.model = model
        self.layers = layers
        self.activations = {layer: None for layer in layers}
        self.gradients = {layer: None for layer in layers}
        hooks = {'forward':{},
                 'backward':{}}   #saving hooks to variables lets us remove them later if we want
        
        for layer_id in layers:
            layer = dict([*self.model.named_modules()])[layer_id]
            hooks['forward'][layer_id] = layer.register_forward_hook(self.save_activations(layer_id)) #execute on forward pass
            hooks['backward'][layer_id] = layer.register_backward_hook(self.save_gradients(layer_id))    #execute on backwards pass

    def save_activations(self, layer_id: str) -> Callable:
        def fn(module, input, output):  #register_hook expects to recieve a function with arguments like this
            #output is what is return by the layer with dim (batch_dim x out_dim), sum across the batch dim
            batch_summed_output = torch.sum(torch.abs(output),dim=0).detach().cpu()
            if self.activations[layer_id] is None:
                self.activations[layer_id] = batch_summed_output
            else:
                self.activations[layer_id] +=  batch_summed_output
        return fn
    
    def save_gradients(self, layer_id: str) -> Callable:
        def fn(module, grad_input, grad_output):
            batch_summed_output = torch.sum(torch.abs(grad_output[0]),dim=0).detach().cpu() #grad_output is a tuple with 'device' as second item
            if self.gradients[layer_id] is None:
                self.gradients[layer_id] = batch_summed_output
            else:
                self.gradients[layer_id] +=  batch_summed_output 
        return fn
    
    def remove_all_hooks(self):
        for hook in self.hooks['forward'].values():
            hook.remove()
        for hook in self.hooks['backward'].values():
            hook.remove()
    

### Running model

In [14]:
target_layers = ["conv_block_3.0","fc6.0"]

model_actgrad_extractor = actgrad_extractor(model, layers=target_layers)
criterion = nn.CrossEntropyLoss()

In [15]:
iter_dataloader = iter(dloader)
iters = len(iter_dataloader)  #if you want to test this out quickly just set this to a small number
print('total batches: ' + str(iters)) 
for it in range(iters):
    if it%10 == 0:
        print(it)
    inputs, target = next(iter_dataloader)
    inputs = inputs.to(device)
    target = target.to(device)

    model.zero_grad()
    
    output = model(inputs)
    
    loss = criterion(output,target)
    loss.backward()
    

actgrad_extractor.remove_all_hooks()

total batches: 196
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


TypeError: remove_all_hooks() missing 1 required positional argument: 'self'

In [16]:
#get average by dividing result by length of dset
activations = model_actgrad_extractor.activations
gradients = model_actgrad_extractor.gradients

for l in target_layers:
    activations[l] /= len(dset)
    gradients[l] /= len(dset)

In [17]:
#scores are just actxgrad! Do with them what you want

scores = {}
for l in target_layers:
    scores[l] = activations[l]*gradients[l]
    print(scores[l].shape)

torch.Size([384, 13, 13])
torch.Size([4096])
